# Data Warehouse Truven QA - Claim ICD Proc

## Initialization

Just loading packages that will be used and initializing connection to GP DB.

In [1]:
import pandas as pd
import sys
import psycopg2
from tqdm import tqdm
sys.path.append('H:/uth_helpers')
from db_utils import get_dsn

In [2]:
connection = psycopg2.connect(get_dsn()+' keepalives=1 keepalives_idle=30 keepalives_interval=10')
connection.autocommit = True

## Row Count and Claim Count

Similar to the member_enrollment_monthly table, the claim_header table has the row count should equal the number of unique claims.

For this table, we extract claim data from the s, o, and f tables from the truven schema. 


In [3]:
query = ''' drop table if exists qa_reporting.dw_truv_claim_icd_proc_counts;
create table qa_reporting.dw_truv_claim_icd_proc_counts
(
    data_source bpchar(4),
    table_id_src text,
    calendar_year int,
    dw_row_count bigint,
    src_row_count bigint,
    row_count_diff bigint,
    row_count_percentage float,
    dw_uth_clm_id_count bigint,
    dw_src_clm_id_count bigint,
    src_clm_count bigint,
    clm_count_diff bigint,
    clm_count_percentage float,
    dw_uth_mbr_id_count bigint,
    dw_src_mbr_id_count bigint,
    src_mbr_count bigint,
    mbr_count_diff bigint,
    mbr_count_percentage float,
    date_generated date
);
'''

with connection.cursor() as cursor:
    cursor.execute(query)

In [4]:
with connection.cursor() as cursor:
    query = f'''
    insert into qa_reporting.dw_truv_claim_icd_proc_counts
    (data_source, calendar_year, dw_row_count, dw_uth_clm_id_count, date_generated)
    select data_source, year, count(a.*), count(distinct uth_claim_id), current_date
    from (
        select data_source, year, uth_claim_id, uth_member_id
        from dw_staging.trum_claim_icd_proc
        where proc_position = 1
    ) a
    group by 1,2
    '''
    
    cursor.execute(query)

    query = f'''
    update qa_reporting.dw_truv_claim_icd_proc_counts b
    set dw_src_clm_id_count = count
    from (
        select data_source, year,  count(distinct claim_id_src) as count 
        from dw_staging.trum_claim_icd_proc
        group by 1,2
    ) a
    where a.year = b.calendar_year
    and a.data_source = b.data_source
    ;
    '''

    cursor.execute(query)

    query = f'''
    update qa_reporting.dw_truv_claim_icd_proc_counts b
    set dw_uth_mbr_id_count = count
    from (
        select data_source, year, count(distinct uth_member_id) as count 
        from dw_staging.trum_claim_icd_proc
        group by 1,2
    ) a
    where a.year = b.calendar_year
    and a.data_source = b.data_source
    ;
    '''

    cursor.execute(query)
    
    query = f'''
    update qa_reporting.dw_truv_claim_icd_proc_counts b
    set dw_src_mbr_id_count = count
    from (
        select data_source, year, count(distinct member_id_src) as count 
        from dw_staging.trum_claim_icd_proc
        group by 1,2
    ) a
    where a.year = b.calendar_year
    and a.data_source = b.data_source
    ;
    '''

    cursor.execute(query)

In [5]:
with connection.cursor() as cursor:
    query = f'''
    insert into qa_reporting.dw_truv_claim_icd_proc_counts
    (data_source, calendar_year, dw_row_count, dw_uth_clm_id_count, date_generated)
    select data_source, year, count(a.*), count(distinct uth_claim_id), current_date
    from (
        select data_source, year, uth_claim_id, uth_member_id
        from dw_staging.truc_claim_icd_proc
        where proc_position = 1
    ) a
    group by 1,2
    '''
    
    cursor.execute(query)

    query = f'''
    update qa_reporting.dw_truv_claim_icd_proc_counts b
    set dw_src_clm_id_count = count
    from (
        select data_source, year,  count(distinct claim_id_src) as count 
        from dw_staging.truc_claim_icd_proc
        group by 1,2
    ) a
    where a.year = b.calendar_year
    and a.data_source = b.data_source
    ;
    '''

    cursor.execute(query)

    query = f'''
    update qa_reporting.dw_truv_claim_icd_proc_counts b
    set dw_uth_mbr_id_count = count
    from (
        select data_source, year, count(distinct uth_member_id) as count 
        from dw_staging.truc_claim_icd_proc
        group by 1,2
    ) a
    where a.year = b.calendar_year
    and a.data_source = b.data_source
    ;
    '''

    cursor.execute(query)
    
    query = f'''
    update qa_reporting.dw_truv_claim_icd_proc_counts b
    set dw_src_mbr_id_count = count
    from (
        select data_source, year, count(distinct member_id_src) as count 
        from dw_staging.truc_claim_icd_proc
        group by 1,2
    ) a
    where a.year = b.calendar_year
    and a.data_source = b.data_source
    ;
    '''

    cursor.execute(query)

claims that are both in the s table and f tables may have the same procedure code. need to consider this

In [6]:
with connection.cursor() as cursor:
    query = '''
with truven_claims as (
        select 'truc' as table_source, year, enrolid, claim_id_derv, svcdate
        from truven.ccaes
        where pproc is not null
        union
        select 'trum', year, enrolid, claim_id_derv, svcdate
        from truven.mdcrs
        where pproc is not null
        union
        select 'truc', year, enrolid, claim_id_derv, svcdate
        from truven.ccaef
        where proc1 is not null
        or proc2 is not null
        or proc3 is not null
        or proc4 is not null
        or proc5 is not null
        or proc6 is not null
        union
        select 'trum', year, enrolid, claim_id_derv, svcdate
        from truven.mdcrf
        where proc1 is not null
        or proc2 is not null
        or proc3 is not null
        or proc4 is not null
        or proc5 is not null
        or proc6 is not null
    ),
truven_claims_counts as (
        select table_source, year as calendar_year, count(*) as src_row_count, count(distinct enrolid) src_mbr_count, count(distinct claim_id_derv) src_clm_count
        from truven_claims
        group by 1,2
    )
update qa_reporting.dw_truv_claim_icd_proc_counts a
set src_row_count = b.src_row_count,
row_count_diff = a.dw_row_count - b.src_row_count,
row_count_percentage = 100. * abs(a.dw_row_count - b.src_row_count) / b.src_row_count,
src_clm_count = b.src_clm_count,
clm_count_diff = a.dw_uth_clm_id_count - b.src_clm_count,
clm_count_percentage = 100. * abs(a.dw_uth_clm_id_count - b.src_clm_count) / b.src_clm_count,
src_mbr_count = b.src_mbr_count,
mbr_count_diff = a.dw_uth_mbr_id_count - b.src_mbr_count,   
mbr_count_percentage = 100. * abs(a.dw_uth_mbr_id_count - b.src_mbr_count) / b.src_mbr_count
from truven_claims_counts b
where a.calendar_year = b.calendar_year
and a.data_source = b.table_source
and a.table_id_src is null
;
    '''

    cursor.execute(query)

In [7]:
query = '''select * from qa_reporting.dw_truv_claim_icd_proc_counts;'''

df = pd.read_sql(query, con=connection)
# df.sort_values(['clm_count_percentage'], ascending=False)

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [8]:
df[df['data_source'] == 'truc'].sort_values(['table_id_src', 'calendar_year'])

,data_source,table_id_src,calendar_year,dw_row_count,src_row_count,row_count_diff,row_count_percentage,dw_uth_clm_id_count,dw_src_clm_id_count,src_clm_count,clm_count_diff,clm_count_percentage,dw_uth_mbr_id_count,dw_src_mbr_id_count,src_mbr_count,mbr_count_diff,mbr_count_percentage,date_generated
11,truc,None,2011,16938600,16427214,511386,3.113042,13842249,13856813,13856813,-14564,0.105104,3161890,3163117,3163117,-1227,0.038791,2023-10-13
10,truc,None,2012,15493783,15110062,383721,2.539506,12734387,12738845,12738845,-4458,0.034995,2616806,2617234,2617234,-428,0.016353,2023-10-13
8,truc,None,2013,12447289,12191354,255935,2.099316,10498685,10505683,10505683,-6998,0.066612,2345888,2346343,2346343,-455,0.019392,2023-10-13
9,truc,None,2014,12543187,12403430,139757,1.126761,10572836,10647806,10647806,-74970,0.704089,2239190,2239892,2239892,-702,0.031341,2023-10-13
7,truc,None,2015,8826563,8758549,68014,0.776544,7402401,7479786,7479786,-77385,1.034588,1413591,1414024,1414024,-433,0.030622,2023-10-13
6,truc,None,2016,10800265,10613136,187129,1.763183,8769793,8775166,8775166,-5373,0.061230,1052074,1052877,1052877,-803,0.076267,2023-10-13
5,truc,None,2017,9751992,9590565,161427,1.683186,8001939,8007245,8007245,-5306,0.066265,902716,903503,903503,-787,0.087105,2023-10-13
4,truc,None,2018,9991637,9838320,153317,1.558366,8189543,8189689,8189689,-146,0.001783,844139,844559,844559,-420,0.049730,2023-10-13
3,truc,None,2019,9474670,9330760,143910,1.542318,7762508,7762081,7762081,427,0.005501,803828,804114,804114,-286,0.035567,2023-10-13
1,truc,None,2020,8868867,8722230,146637,1.681187,7234655,7234220,7234220,435,0.006013,713498,713788,713788,-290,0.040628,2023-10-13


In [9]:
df[df['data_source'] == 'trum'].sort_values(['table_id_src', 'calendar_year'])

,data_source,table_id_src,calendar_year,dw_row_count,src_row_count,row_count_diff,row_count_percentage,dw_uth_clm_id_count,dw_src_clm_id_count,src_clm_count,clm_count_diff,clm_count_percentage,dw_uth_mbr_id_count,dw_src_mbr_id_count,src_mbr_count,mbr_count_diff,mbr_count_percentage,date_generated
23,trum,None,2011,6869139,6792563,76576,1.127351,5153121,5155807,5155807,-2686,0.052097,674041,674400,674400,-359,0.053233,2023-10-13
22,trum,None,2012,6389031,6304917,84114,1.334102,4871518,4873037,4873037,-1519,0.031172,641996,642216,642216,-220,0.034256,2023-10-13
21,trum,None,2013,5600819,5514188,86631,1.571056,4366754,4370012,4370012,-3258,0.074554,580232,580550,580550,-318,0.054776,2023-10-13
20,trum,None,2014,4982278,4928132,54146,1.098712,3890784,3907119,3907119,-16335,0.418083,499101,499352,499352,-251,0.050265,2023-10-13
19,trum,None,2015,3607357,3571995,35362,0.989979,2823690,2835039,2835039,-11349,0.400312,329893,329944,329944,-51,0.015457,2023-10-13
18,trum,None,2016,4202994,4159888,43106,1.036230,3272811,3272796,3272796,15,0.000458,257008,257095,257095,-87,0.033840,2023-10-13
15,trum,None,2017,2954214,2925553,28661,0.979678,2357679,2357313,2357313,366,0.015526,162026,162098,162098,-72,0.044418,2023-10-13
12,trum,None,2018,1978993,1962662,16331,0.832084,1608103,1607341,1607341,762,0.047407,98184,98200,98200,-16,0.016293,2023-10-13
17,trum,None,2019,3821416,3791337,30079,0.793361,3010823,3010791,3010791,32,0.001063,168943,168958,168958,-15,0.008878,2023-10-13
16,trum,None,2020,3981906,3761230,220676,5.867123,2964686,2964897,2964897,-211,0.007117,159023,159054,159054,-31,0.019490,2023-10-13


## ICD Procedure Codes

In [10]:
query = '''drop table if exists qa_reporting.dw_truv_icd_proc_counts;

select data_source, year, proc_cd, count(*) as proc_count
into qa_reporting.dw_truv_icd_proc_counts
from dw_staging.trum_claim_icd_proc
group by 1,2,3;

insert into qa_reporting.dw_truv_icd_proc_counts
select data_source, year, proc_cd, count(*) as proc_count
from dw_staging.truc_claim_icd_proc
group by 1,2,3;
'''

with connection.cursor() as cursor:
    cursor.execute(query)

In [11]:
icd_proc_cd_df = pd.read_sql('select * from qa_reporting.dw_truv_icd_proc_counts;', con=connection)
icd_proc_cd_df

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,data_source,year,proc_cd,proc_count
0,trum,2013,3401,148
1,trum,2020,04QD0ZZ,37
2,trum,2019,0GSM0ZZ,106
3,trum,2017,BF35ZZZ,19
4,trum,2021,0JCH3ZZ,6
...,...,...,...,...
294036,truc,2011,88160,1
294037,truc,2019,DD010ZZ,25
294038,truc,2019,0UU447Z,1
294039,truc,2012,9325,31


In [12]:
query = '''
select a.*
from qa_reporting.dw_truv_icd_proc_counts a
left join reference_tables.ref_cms_icd_pcs_codes b
on a.proc_cd = cd_value
where b.cd_value is null;
'''

invalid_proc_df = pd.read_sql(query, con=connection)
invalid_proc_df

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,data_source,year,proc_cd,proc_count
0,trum,2020,11721,9
1,trum,2020,66850,1
2,truc,2011,37735,1
3,truc,2020,82652,1
4,truc,2011,83090,93
...,...,...,...,...
16483,truc,2012,J3101,1
16484,truc,2016,99070,1
16485,trum,2011,82365,1
16486,trum,2020,83880,1009


In [13]:
invalid_proc_df.groupby(['data_source', 'year'])['proc_count'].sum()

data_source  year
truc         2011    444830
             2012    254364
             2013     74004
             2014     32851
             2015     32275
             2016      2052
             2017       171
             2018        88
             2019        66
             2020     16815
             2021        88
             2022       742
trum         2011      1235
             2012     12854
             2013      2817
             2014      9815
             2015      6942
             2016        21
             2017         3
             2018         9
             2019         1
             2020    295011
             2021         1
             2022       215
Name: proc_count, dtype: int64

In [14]:
proc_comp_df = pd.DataFrame({'overall_proc_count': icd_proc_cd_df.groupby(['data_source', 'year'])['proc_count'].sum(),
                            'invalid_proc_count': invalid_proc_df.groupby(['data_source', 'year'])['proc_count'].sum(),
                            'valid_proc_count': icd_proc_cd_df.groupby(['data_source', 'year'])['proc_count'].sum() - invalid_proc_df.groupby(['data_source', 'year'])['proc_count'].sum()})
proc_comp_df.loc[proc_comp_df['valid_proc_count'].isna(),'valid_proc_count'] = proc_comp_df.loc[proc_comp_df['valid_proc_count'].isna(),'overall_proc_count']
proc_comp_df['valid_proc_count'] =  proc_comp_df['valid_proc_count'].astype(int)
proc_comp_df['invalid_to_valid_percent'] = 100. * proc_comp_df['invalid_proc_count'] / proc_comp_df['valid_proc_count']
proc_comp_df  

overall_proc_count  invalid_proc_count  valid_proc_count  \
data_source year                                                             
truc        2011            19764426              444830          19319596   
            2012            18146219              254364          17891855   
            2013            14713996               74004          14639992   
            2014            15475067               32851          15442216   
            2015            10780067               32275          10747792   
            2016            12245154                2052          12243102   
            2017            11088552                 171          11088381   
            2018            11549780                  88          11549692   
            2019            10979629                  66          10979563   
            2020            10299837               16815          10283022   
            2021            10860232                  88          10860144   
            2022             8988748                 742           8988006   
trum        2011             7584998                1235           7583763   
            2012             7085833               12854           7072979   
            2013             6294239                2817           6291422   
            2014             5726259                9815           5716444   
            2015             4076713                6942           4069771   
            2016             4606142                  21           4606121   
            2017             3191473                   3           3191470   
            2018             2127775                   9           2127766   
            2019             4145584                   1           4145583   
            2020             4297631              295011           4002620   
            2021             3318224                   1           3318223   
            2022             3515350                 215           3515135   

                  invalid_to_valid_percent  
data_source year                            
truc        2011                  2.302481  
            2012                  1.421675  
            2013                  0.505492  
            2014                  0.212735  
            2015                  0.300294  
            2016                  0.016760  
            2017                  0.001542  
            2018                  0.000762  
            2019                  0.000601  
            2020                  0.163522  
            2021                  0.000810  
            2022                  0.008255  
trum        2011                  0.016285  
            2012                  0.181734  
            2013                  0.044775  
            2014                  0.171698  
            2015                  0.170575  
            2016                  0.000456  
            2017                  0.000094  
            2018                  0.000423  
            2019                  0.000024  
            2020                  7.370447  
            2021                  0.000030  
            2022                  0.006116

In [15]:
invalid_proc_df.groupby(['data_source', 'year']).max()

proc_cd  proc_count
data_source year                     
truc        2011    V5275       22072
            2012    V5299       14349
            2013    V2744        3325
            2014    V2632       16975
            2015    S5001       17055
            2016    S5001         261
            2017    L8699          54
            2018    Q0144          18
            2019    J7999           7
            2020    V5257        1008
            2021    U0004           5
            2022  XW0H7X8         220
trum        2011    V2632         127
            2012    V2787         546
            2013    V2787         911
            2014    V2787        4475
            2015    V2632        3247
            2016    J2930           3
            2017    99284           2
            2018    J8499           1
            2019    36581           1
            2020    V5298       12579
            2021      320           1
            2022  XW03358          76

## Procedure Code Position

In [16]:
query = '''drop table if exists qa_reporting.dw_truv_proc_position;
select data_source, year, proc_position, count(distinct uth_claim_id) as claim_count
into qa_reporting.dw_truv_proc_position
from dw_staging.trum_claim_icd_proc
group by 1,2,3;

insert into qa_reporting.dw_truv_proc_position
select data_source, year, proc_position, count(distinct uth_claim_id) as claim_count
from dw_staging.truc_claim_icd_proc
group by 1,2,3;
'''
with connection.cursor() as cursor:
    cursor.execute(query)

In [17]:
proc_position_df = pd.read_sql('select * from qa_reporting.dw_truv_proc_position;', con=connection)
proc_position_df.sort_values(['data_source', 'year', 'proc_position'])

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,data_source,year,proc_position,claim_count
122,truc,2011,1,13842249
13,truc,2011,2,1348799
143,truc,2011,3,687143
12,truc,2011,4,338345
53,truc,2011,5,199769
...,...,...,...,...
136,trum,2022,2,100040
65,trum,2022,3,67620
42,trum,2022,4,45020
131,trum,2022,5,31243


In [18]:
sorted(proc_position_df['proc_position'].unique())

[1, 2, 3, 4, 5, 6]

Checking if the counts for the proc_position are correct. The higher the proc_position is, the less counts there should be. If we sort the counts of the proc_position by year and assign their order, this value should match with the proc_position value.

In [19]:
proc_position_df['row_rank'] = proc_position_df.sort_values(['data_source', 'year', 'claim_count'], ascending=[True, True, False]).groupby(['data_source', 'year']).cumcount()+1
proc_position_df['position_check'] = proc_position_df['row_rank'] == proc_position_df['proc_position']
proc_position_df[~proc_position_df['position_check']]

,data_source,year,proc_position,claim_count,row_rank,position_check


## ICD Version

In [20]:
query = '''drop table if exists qa_reporting.dw_truv_icd_proc_version_count;
select data_source, year, icd_version, count(distinct uth_claim_id) as claim_count
into qa_reporting.dw_truv_icd_proc_version_count
from dw_staging.trum_claim_icd_proc
group by 1,2,3;

insert into qa_reporting.dw_truv_icd_proc_version_count
select data_source, year, icd_version, count(distinct uth_claim_id) as claim_count
from dw_staging.truc_claim_icd_proc
group by 1,2,3;
'''
with connection.cursor() as cursor:
    cursor.execute(query)

In [21]:
icd_version_df = pd.read_sql('select * from qa_reporting.dw_truv_icd_proc_version_count;', con=connection)
icd_version_df.sort_values(['data_source', 'year', 'icd_version'])

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,data_source,year,icd_version,claim_count
23,truc,2011,None,13853100
19,truc,2012,None,12739100
18,truc,2013,None,10506175
4,truc,2014,None,10647161
29,truc,2015,0,2085203
41,truc,2015,9,3664353
27,truc,2015,None,2160218
5,truc,2016,0,8353147
28,truc,2016,9,1255
32,truc,2016,None,1052777
